In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from itertools import product
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, Activation, Dropout
from tensorflow_model_optimization.quantization.keras import vitis_quantize


shape_range =range(160,480,32)

for shape_x in shape_range:
    
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0


    inputs = Input(shape=(shape_x, shape_y, 1))
    for step in range(1,11):
        zeroPadding1 = ZeroPadding2D(padding=1)(inputs)
        conv1_kernel_num = random.randrange(32,64,2)
        conv1 = Conv2D(conv1_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding1)
        pool1 = MaxPooling2D(pool_size=2)(conv1)

        zeroPadding2 = ZeroPadding2D(padding=1)(pool1)
        conv2_kernel_num = random.randrange(conv1_kernel_num,128,2)
        conv2 = Conv2D(conv2_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding2)
        pool2 = MaxPooling2D(pool_size=2)(conv2)

        zeroPadding3 = ZeroPadding2D(padding=1)(pool2)
        conv3_kernel_num = random.randrange(conv2_kernel_num,256,2)
        conv3 = Conv2D(conv3_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding3)

        zeroPadding4 = ZeroPadding2D(padding=1)(conv3)
        conv4_kernel_num = random.randrange(conv3_kernel_num,256,2)
        conv4 = Conv2D(conv4_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding4)
        pool3 = MaxPooling2D(pool_size=2)(conv4)

        zeroPadding5 = ZeroPadding2D(padding=1)(pool3)
        conv5_kernel_num = random.randrange(conv4_kernel_num,512,2)
        conv5 = Conv2D(conv5_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding5)

        zeroPadding6 = ZeroPadding2D(padding=1)(conv5)
        conv6_kernel_num = random.randrange(conv5_kernel_num,512,2)
        conv6 = Conv2D(conv6_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding6)
        pool4 = MaxPooling2D(pool_size=2)(conv6)

        zeroPadding7 = ZeroPadding2D(padding=1)(pool4)
        conv7_kernel_num = random.randrange(conv5_kernel_num,512,2)
        conv7 = Conv2D(conv7_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding7)

        zeroPadding8 = ZeroPadding2D(padding=1)(conv7)
        conv8_kernel_num = random.randrange(conv7_kernel_num,512,2)
        conv8 = Conv2D(conv8_kernel_num, (3,3), padding='same', activation='relu')(zeroPadding8)
        pool5 = MaxPooling2D(pool_size=2)(conv8)

        conv9_kernel_num = random.randrange(conv8_kernel_num,4096,2)
        conv9 = Conv2D(conv9_kernel_num, (7,7), padding='same', activation='relu')(pool5)
        drop1 = Dropout(0.5)(conv9)

        conv10_kernel_num = random.randrange(conv9_kernel_num,4096,2)
        conv10 = Conv2D(conv10_kernel_num, (1,1), padding='same', activation='relu')(pool5)
        drop2 = Dropout(0.5)(conv10)

        conv11 = Conv2D(10, (1,1))(drop2)

        flat = Flatten()(conv11)

        outputs = Activation('softmax')(flat)


        model = Model(inputs=inputs, outputs=outputs)
        model.summary()
        model.save('./vgg11_h5/vgg11_{input_x}_{input_y}_{step}.h5'.format(input_x=shape_x,input_y=shape_y,step=step))
#         model.compile(optimizer='adam',
#                   loss="sparse_categorical_crossentropy", 
#                   metrics=['accuracy'])
#         history = model.fit(X_train, Y_train, epochs=1)
#         model.get_weights()[0].dtype
        quantizer = vitis_quantize.VitisQuantizer(model)
        quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
#         quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
        quantized_model.save('quantized.h5')
        !vai_c_tensorflow2 \
                --model ./quantized.h5 \
                --arch ./arch.json \
                --output_dir ./xmodel_vgg11 \
                --net_name deploy
        os.rename("./xmodel_vgg11/deploy.xmodel","./xmodel_vgg11/vgg11_{input_x}_{input_y}_{step}.xmodel".format(input_x=shape_x,input_y=shape_y,step=step))

2022-04-12 02:29:29.793445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-12 02:29:29.793460: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-12 02:29:31.033081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2022-04-12 02:29:31.033106: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 1)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 162, 162, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 162, 162, 50)      500       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 81, 81, 50)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 83, 83, 50)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 83, 83, 94)        42394     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 41, 94)        0     

2022-04-12 02:29:39.243062: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 [==============================] - 5s 5s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :100%|█| 51/51 [00:00<00:00, 856.52it/s]             
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B2304_MAX_BG2
[UNILOG][INFO] Graph name: model, with op num: 91
[UNILOG

[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/deploy.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is 4238f6f204ee52f1188ea81305597e60, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 1)]     0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 162, 162, 1)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 162, 162, 34)      340       
_________________________________________________________________
max_p

...

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 231ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[INFO] generate xmodel     :100%|█| 51/51 [00:00<00:00, 1576.38it/s]            
[INFO] dump xmodel: /tmp/deploy_org.xmodel
[UNILOG][INFO] Compile mode: dpu
[UNILOG][INFO] Debug mode: function
[UNILOG][INFO] Target architecture: DPUCZDX8G_ISA0_B2304_MAX_BG2
[UNILOG][INFO] Graph name: model_94, with op num: 91
[UNILOG][INFO] Begin to compile...
[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/deploy.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is 8ccfa1655b640c8536cbdecd9a7f518e, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 448, 448, 1)]    

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 211ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 9s 9s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel     :

[UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
[UNILOG][INFO] Compile done.
[UNILOG][INFO] The meta json is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/meta.json"
[UNILOG][INFO] The compiled xmodel is saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/deploy.xmodel"
[UNILOG][INFO] The compiled xmodel's md5sum is d5aa40530ad566de6c09f9a2fe5cc80c, and has been saved to "/workspace/DPU-PYNQ/host/./xmodel_vgg11/md5sum.txt"
Model: "model_98"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 448, 448, 1)]     0         
_________________________________________________________________
zero_padding2d_784 (ZeroPadd (None, 450, 450, 1)       0         
_________________________________________________________________
conv2d_1078 (Conv2D)         (None, 450, 450, 32)      320       
_________________________________________________________________
max_

[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 2s 222ms/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 10s 10s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras version: 2.6.0
[INFO] Tensorflow Keras model type: functional
[INFO] generate xmodel    